In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

import numpy as np

data = np.genfromtxt('zcash-shielded-data-811497.csv', delimiter=',', skip_header=1)

In [ ]:
COIN = 100000000

def sum_chunk(x, chunk_size, axis=-1):
    shape = x.shape
    if axis < 0:
        axis += x.ndim
    shape = shape[:axis] + (-1, chunk_size) + shape[axis+1:]
    x = x.reshape(shape)
    return x.sum(axis=axis+1)

from_height = 720000
to_height = 811000
chunk = 1000

# Slice the heights
height = sum_chunk(data[from_height:to_height,0], chunk)

num_types = sum_chunk(data[from_height:to_height,1:4].transpose(), chunk)
percent_types = num_types /  num_types.sum(axis=0).astype(float) * 100

num_migrating = data[from_height:to_height,10]
num_mixed = data[from_height:to_height,9] - num_migrating
num_shielding = data[from_height:to_height,5]
num_unshielding = data[from_height:to_height,6] - num_migrating - num_mixed
num_shielded = data[from_height:to_height,3] - data[from_height:to_height,5] - data[from_height:to_height,6]

num_shielded = sum_chunk(num_shielded, chunk)
num_shielding = sum_chunk(num_shielding, chunk)
num_unshielding = sum_chunk(num_unshielding, chunk)
num_mixed = sum_chunk(num_mixed, chunk)
num_migrating = sum_chunk(num_migrating, chunk)

# percent_split = num_split / num_split.sum(axis=0).astype(float) * 100

sprout_shielded = data[from_height:to_height,11]
sapling_shielded = data[from_height:to_height,12]
sprout_unshielded = data[from_height:to_height,13]
sapling_unshielded = data[from_height:to_height,14]
print(sprout_shielded.max() / COIN)
print(sprout_unshielded.max() / COIN)
print(sapling_shielded.max() / COIN)
print(sapling_unshielded.max() / COIN)

sprout_shielded = sum_chunk(sprout_shielded, chunk)
sprout_unshielded = sum_chunk(sprout_unshielded, chunk)
sapling_shielded = sum_chunk(sapling_shielded, chunk)
sapling_unshielded = sum_chunk(sapling_unshielded, chunk)

# Rescale for chunk size
height = height / chunk
num_types = num_types / chunk
num_shielded = num_shielded / chunk
num_shielding = num_shielding / chunk
num_unshielding = num_unshielding / chunk
num_mixed = num_mixed / chunk
num_migrating = num_migrating / chunk
sprout_shielded = sprout_shielded / (chunk * COIN)
sprout_unshielded = sprout_unshielded / (chunk * COIN)
sapling_shielded = sapling_shielded / (chunk * COIN)
sapling_unshielded = sapling_unshielded / (chunk * COIN)

# Select the types char
chart_types = num_types.transpose()

sprout = chart_types[:,2]
sapling = chart_types[:,1]
transparent = chart_types[:,0]

In [ ]:
fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(1, 1, 1)

ax.stackplot(height, sprout, sapling, transparent, labels=['Involves Sprout', 'Sapling + Transparent', 'Transparent-only'])
ax.set_title('Transaction types mined in the Zcash main chain')
ax.set_xlabel('Height')
ax.set_ylabel('Transactions per block (%d-block average)' % chunk)
ax.legend(loc='upper right')
ax.grid()
ax.set_yscale('log')
plt.xticks(rotation=45)
plt.margins(x=0)
plt.savefig('tx-types-%d-%d-%d.png' % (from_height, to_height, chunk))

In [ ]:
fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(1, 1, 1)

# ax.fill_between(height, sprout_shielded, label='Sprout Shielding')
# ax.fill_between(height, sprout_unshielded * -1, label='Sprout Unshielding')
# ax.fill_between(height, sapling_shielded, label='Sprout Shielding')
# ax.fill_between(height, sapling_unshielded * -1, label='Sprout Unshielding')
ax.stackplot(height, sprout_shielded, sapling_shielded, labels=['Sprout Shielding', 'Sapling Shielding'])
ax.stackplot(height, sprout_unshielded * -1, sapling_unshielded * -1, labels=['Sprout Unshielding', 'Sapling Unshielding'])

ax.set_title('ZEC moving into and out of the shielded value pools')
ax.set_xlabel('Height')
ax.set_ylabel('ZEC amount (%d-block average)' % chunk)
# ax.set_ylim(-10, 10)
ax.legend(loc='lower right')
ax.grid()
plt.xticks(rotation=45)
plt.margins(x=0)
plt.savefig('tx-values-%d-%d-%d.png' % (from_height, to_height, chunk))

In [ ]:
fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(1, 1, 1)

ax.stackplot(height, num_mixed, num_migrating, num_shielded, num_unshielding, num_shielding, labels=['Mixed', 'Migrating', 'Shielded', 'Unshielding', 'Shielding'])
ax.set_title('Sprout transaction types mined in the Zcash main chain')
ax.set_xlabel('Height')
ax.set_ylabel('Transactions per block (%d-block average)' % chunk)
ax.legend(loc='upper right')
ax.grid()
# ax.set_yscale('log')
plt.xticks(rotation=45)
plt.margins(x=0)
plt.savefig('tx-sprout-splits-%d-%d-%d.png' % (from_height, to_height, chunk))